In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
import numba
from scipy.io import wavfile
from IPython.display import Audio

In [ ]:
# Simulation settings
Nx = 101
Nt = 500000
dt = 5e-6

# Sampling settings
samplerate = 44000
downRateFactor = 5

# Guitar settings
scaleLength = 0.65
neckLength = 0.43
numberOfFrets = 19
amplitude = 0.002

# Wave Equation settings
l=5e-5
gamma=5e-5

In [ ]:
def noteToFrequency(note):
    if note == 'E':
        return 82
    elif note == 'A':
        return 108
    elif note == 'D':
        return 146.83
    elif note == 'G':
        return 193
    elif note == 'B':
        return 246
    elif note == 'e':
        return 331
    
def initialCondition(initialLength, totalLength):
    x = np.linspace(0, totalLength, Nx)
    sigma = 0.1  
    y = np.exp(-(x - initialLength)**2 / (2 * sigma**2))
    return amplitude* y / np.max(y)  


def notePosition(noteNumber):
    n = numberOfFrets - noteNumber
    noteLength = scaleLength - neckLength
    for i in range(n):
        noteLength += 1/100 + i/1000 + 2/1000
        if i/10 >= 1.5:
            noteLength += 1/1000
    return noteLength - 1/1000


In [28]:
@numba.jit("f8[:,:](f8[:,:], i8, i8, f8, f8, f8, f8, f8)", nopython=True, nogil=True)
def computeSolution(d, times, length, dt, dx, l, gamma, c):
    for t in range(1, times-1):
        for i in range(2, length-2):
            outer_fact = (1/(c**2 * dt**2) + gamma/(2*dt))**(-1)
            p1 = 1/dx**2 * (d[t][i-1] - 2*d[t][i] + d[t][i+1])
            p2 = 1/(c**2 * dt**2) * (d[t-1][i] - 2*d[t][i])
            p3 = gamma/(2*dt) * d[t-1][i]
            p4 = l**2 / dx**4 * (d[t][i+2] - 4*d[t][i+1] + 6*d[t][i] - 4*d[t][i-1] + d[t][i-2])
            d[t+1][i] = outer_fact * (p1 - p2 + p3 - p4)
    return d

def fourierAnalysis(n, sol):
    sin_arr = np.sin(n*np.pi*np.linspace(0,1,Nx))
    return np.multiply(sol, sin_arr).sum(axis=1)

def createNote(noteNumber, string):
    c = 2*scaleLength*noteToFrequency(string)
    noteLength = notePosition(noteNumber)
    dx = noteLength/(Nx-1)
    y0 = initialCondition(scaleLength/4, noteLength)

    sol = np.zeros((Nt, Nx))
    sol[0] = y0
    sol[1] = y0
    sol = computeSolution(sol, Nt, Nx, dt, dx, l, gamma, c)
    
    hms = [fourierAnalysis(n, sol) for n in range(10)]
    return sum(hms)[::downRateFactor], sol

def playNote(note, name='note'):
    tot = note.astype(np.float32)
    wavfile.write(f'{name}.wav',int(1/(dt*downRateFactor)),tot)
    return Audio(f'{name}.wav')

def createChord(notes):
    chord = np.zeros(int(Nt/downRateFactor))
    for n in notes:
        note, _ = createNote(*n)
        chord += note
    return chord
    
def createChordAndmakeChordAnimation(noteNumbers, stringNames, name='test'):
    strings = []
    chord = np.zeros(int(Nt/downRateFactor))
    for i in range(6):
        if stringNames[i] == 'N':
            string = np.zeros((Nt, Nx))
        else: 
            string = np.zeros((Nt, int(Nx * scaleLength / notePosition(noteNumbers[i]))))
            note, solution = createNote(noteNumbers[i], stringNames[i])
            string[:, :Nx] = solution
            chord += note
        string[:, :] += i * 2 * amplitude
        strings.append(string)
    playNote(chord, name)

    fps = 30
    def animate(i):
        ax.clear()
        for n, string in enumerate(strings):
            stringFrame = string[int(i/(dt*fps))]
            ax.plot(notePosition(noteNumbers[n]) / scaleLength * np.arange(len(stringFrame)), stringFrame, color='white')
        ax.set_ylim(-amplitude, 12*amplitude)
        ax.axis('off')
        ax.set_facecolor('black')
    fig, ax = plt.subplots(1,1,figsize=((25,12)))
    ax.set_ylim(-amplitude, 12*amplitude)
    ani = animation.FuncAnimation(fig, animate, frames=int(Nt*dt*fps), interval=1000*dt)
    ani.save(f'{name}.mp4',writer='ffmpeg',fps=fps,savefig_kwargs={'facecolor':'black'})

In [ ]:
# Smoke on the Water
createChordAndmakeChordAnimation([0,0,0,0,0,0],['N','N','N','N','N','N'],'Silence-Long')
createChordAndmakeChordAnimation([0,0,0,5,5,0],['N','N','N','D','A','N'],'DA55-Long')
createChordAndmakeChordAnimation([0,0,0,5,5,0],['N','N','N','D','A','N'],'DA55')
createChordAndmakeChordAnimation([0,0,3,3,0,0],['N','N','G','D','N','N'],'GD33')
createChordAndmakeChordAnimation([0,0,5,5,0,0],['N','N','G','D','N','N'],'GD55')
createChordAndmakeChordAnimation([0,0,6,6,0,0],['N','N','G','D','N','N'],'GD66')
createChordAndmakeChordAnimation([0,0,0,0,0,0],['N','N','N','N','N','E'],'E0')
createChordAndmakeChordAnimation([0,0,0,0,0,1],['N','N','N','N','N','E'],'E1')
createChordAndmakeChordAnimation([0,0,0,0,0,2],['N','N','N','N','N','E'],'E2')
createChordAndmakeChordAnimation([0,0,0,0,0,3],['N','N','N','N','N','E'],'E3')
createChordAndmakeChordAnimation([0,1,0,0,0,3],['N','B','G','N','N','E'],'BGE1')
createChordAndmakeChordAnimation([0,0,0,0,0,3],['N','N','G','D','N','E'],'GDE3')
createChordAndmakeChordAnimation([0,0,3,3,0,3],['N','N','G','D','N','E'],'GDE333')
createChordAndmakeChordAnimation([0,1,0,0,0,3],['N','B','G','N','N','E'],'BGE1')
createChordAndmakeChordAnimation([0,2,1,0,0,3],['N','B','G','N','N','E'],'BGE213')
createChordAndmakeChordAnimation([0,0,0,0,3,0],['N','N','N','N','A','N'],'A3')
createChordAndmakeChordAnimation([0,0,3,3,1,0],['N','N','G','D','A','N'],'GDA331')
createChordAndmakeChordAnimation([0,0,0,0,1,0],['N','N','N','N','A','N'],'A1')


In [ ]:
# Across the Universe
createChordAndmakeChordAnimation([0,2,2,4,4,2],['N','B','G','D','A','E'],'Fm#1')
createChordAndmakeChordAnimation([0,2,4,4,4,2],['N','B','G','D','A','E'],'Fm#2')
createChordAndmakeChordAnimation([0,2,2,2,0,0],['N','B','G','D','A','N'],'A1')
createChordAndmakeChordAnimation([0,2,4,2,0,0],['N','B','G','D','A','N'],'A2')
createChordAndmakeChordAnimation([2,3,2,0,0,0],['e','B','G','D','N','N'],'D')
createChordAndmakeChordAnimation([2,3,4,4,1,0],['e','B','G','D','A','N'],'Bm')
createChordAndmakeChordAnimation([2,2,2,4,4,2],['e','B','G','D','A','E'],'F#m')
createChordAndmakeChordAnimation([0,3,0,2,2,0],['e','B','G','D','A','E'],'Em7')